In [1]:
# image features를 .pkl 파일에 저장한다. (test)

In [4]:

# 1. 파일 경로 불러오기
import os

dir_num=769
data_dir="/home/gpuadmin/dev/Trajectory_Prediction/traffino/datasets/waterloo/img/train/"+ str(dir_num)+"/"

all_files = [os.path.join(data_dir, path) for path in os.listdir(data_dir) if path[0] != "." and path.endswith(".jpg")] # 이미지 파일 경로

print(f"폴더 내 파일 개수: {len(all_files)}")
all_files=sorted(all_files)
print(all_files)



폴더 내 파일 개수: 727
['/home/gpuadmin/dev/Trajectory_Prediction/traffino/datasets/waterloo/img/train/769/769_frame_0.jpg', '/home/gpuadmin/dev/Trajectory_Prediction/traffino/datasets/waterloo/img/train/769/769_frame_1.jpg', '/home/gpuadmin/dev/Trajectory_Prediction/traffino/datasets/waterloo/img/train/769/769_frame_10.jpg', '/home/gpuadmin/dev/Trajectory_Prediction/traffino/datasets/waterloo/img/train/769/769_frame_100.jpg', '/home/gpuadmin/dev/Trajectory_Prediction/traffino/datasets/waterloo/img/train/769/769_frame_101.jpg', '/home/gpuadmin/dev/Trajectory_Prediction/traffino/datasets/waterloo/img/train/769/769_frame_102.jpg', '/home/gpuadmin/dev/Trajectory_Prediction/traffino/datasets/waterloo/img/train/769/769_frame_103.jpg', '/home/gpuadmin/dev/Trajectory_Prediction/traffino/datasets/waterloo/img/train/769/769_frame_104.jpg', '/home/gpuadmin/dev/Trajectory_Prediction/traffino/datasets/waterloo/img/train/769/769_frame_105.jpg', '/home/gpuadmin/dev/Trajectory_Prediction/traffino/datasets/w

In [5]:
train_path=data_dir
print(train_path)

/home/gpuadmin/dev/Trajectory_Prediction/traffino/datasets/waterloo/img/train/769/


In [6]:
from typing import Tuple
class CreateArg():
    def __init__(self):
        # Dataset options
        self.dataset_name = 'waterloo'
        self.delim = '\t'
        self.loader_num_workers = 4
        self.obs_len = 8
        self.pred_len = 8
        self.skip = 1
        # Optimization
        self.batch_size = 4                 #### batch_size test !!!
        self.num_iterations = 1000 # 10000
        self.num_epochs = 50 # 200                     
        # Model Options
        self.embedding_dim = 64
        self.num_layers = 1
        self.dropout = 0.0
        self.batch_norm = 0 
        self.mlp_dim = 1024
        # self.default_backbone= 'resnet18'
                                    
        # Generator Options
        self.encoder_h_dim_g = 64
        self.decoder_h_dim_g = 128
        self.noise_dim : Tuple[int] = (0, 0) # default=None # type=int_tuple
        self.noise_type = 'gaussian'
        self.noise_mix_type = 'ped'
        self.clipping_threshold_g = 0 # type=float
        self.g_learning_rate = 5e-4 # type=float 
        self.g_steps = 1

        # Pooling Options
        self.pooling_type = 'pool_net' 
        self.pool_every_timestep = 1 # type=bool_flag

        # Pool Net Option
        self.bottleneck_dim = 1024 # type=int

        # Social Pooling Options
        self.neighborhood_size = 1024 # type=float
        self.grid_size = 8 # type=int

        # Discriminator Options
        self.d_type = 'local' # type=str
        self.encoder_h_dim_d = 64 # type=int
        self.d_learning_rate = 5e-4 # type=float
        self.d_steps = 2 # type=int        
        self.clipping_threshold_d = 0 # type=float  

        # Loss Options
        self.l2_loss_weight = 0 # type=float 
        self.best_k = 1 # type=int 

        # Output
        self.output_dir = os.getcwd()+ '/output/img_output'
        self.print_every = 5 # type=int
        self.checkpoint_every = 100 # type=int
        self.checkpoint_name = 'checkpoint_traffic' 
        self.checkpoint_start_from = None
        self.restore_from_checkpoint = 1 # type=int
        self.num_samples_check = 5000 # type=int        

        # Misc
        self.use_gpu = 1 # type=int
        self.timing = 0 # type=int
        self.gpu_num = "1" # type=str   


args = CreateArg() 

In [7]:

from loader_image import img_loader #####
import logging
logger = logging.getLogger(__name__)

import torchvision.transforms as T
# transforms = T.Compose([T.ToTensor()]) # Normalize the pixels and convert to tensor.

logger.info("Initializing train dataset")

train_dset, train_loader = img_loader(args, train_path) # train_dset은 TrajectoryDataset, train_loader는 DataLoader


In [8]:
print(train_dset)
print(train_loader)

In [9]:
print(train_dset.tensor_list)
print(type(train_dset.tensor_list))
print(len(train_dset.tensor_list))


[tensor([[[0.0000, 0.0000, 0.0078,  ..., 0.0078, 0.0078, 0.0039],
         [0.0039, 0.0000, 0.0118,  ..., 0.0000, 0.0000, 0.0000],
         [0.0039, 0.0000, 0.0118,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.0314, 0.1922, 0.1294,  ..., 0.2902, 0.1725, 0.0039],
         [0.0000, 0.0039, 0.0000,  ..., 0.2863, 0.1765, 0.0039],
         [0.0235, 0.0118, 0.0000,  ..., 0.2980, 0.1882, 0.0196]],

        [[0.0000, 0.0000, 0.0039,  ..., 0.0078, 0.0078, 0.0039],
         [0.0039, 0.0000, 0.0078,  ..., 0.0039, 0.0039, 0.0078],
         [0.0039, 0.0000, 0.0078,  ..., 0.0157, 0.0235, 0.0275],
         ...,
         [0.0353, 0.1961, 0.1333,  ..., 0.2824, 0.1725, 0.0039],
         [0.0039, 0.0078, 0.0039,  ..., 0.2784, 0.1765, 0.0039],
         [0.0235, 0.0118, 0.0000,  ..., 0.2902, 0.1882, 0.0196]],

        [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0039, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0039, 0.0000, 0.0000,  ..., 0.0118, 0.0078, 0

In [10]:
print(train_dset.tensor_list[0].shape)
print(train_dset.tensor_list[0])

torch.Size([3, 360, 640])
tensor([[[0.0000, 0.0000, 0.0078,  ..., 0.0078, 0.0078, 0.0039],
         [0.0039, 0.0000, 0.0118,  ..., 0.0000, 0.0000, 0.0000],
         [0.0039, 0.0000, 0.0118,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.0314, 0.1922, 0.1294,  ..., 0.2902, 0.1725, 0.0039],
         [0.0000, 0.0039, 0.0000,  ..., 0.2863, 0.1765, 0.0039],
         [0.0235, 0.0118, 0.0000,  ..., 0.2980, 0.1882, 0.0196]],

        [[0.0000, 0.0000, 0.0039,  ..., 0.0078, 0.0078, 0.0039],
         [0.0039, 0.0000, 0.0078,  ..., 0.0039, 0.0039, 0.0078],
         [0.0039, 0.0000, 0.0078,  ..., 0.0157, 0.0235, 0.0275],
         ...,
         [0.0353, 0.1961, 0.1333,  ..., 0.2824, 0.1725, 0.0039],
         [0.0039, 0.0078, 0.0039,  ..., 0.2784, 0.1765, 0.0039],
         [0.0235, 0.0118, 0.0000,  ..., 0.2902, 0.1882, 0.0196]],

        [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0039, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0039, 0.0000, 0.0000

In [11]:
## feature 추출을 위한 simple conv layer

In [12]:
# 2. test : 한 개의 이미지로부터 불러오기
import torch
import torch.nn as nn
class conv2DBatchNormRelu(nn.Module):
    def __init__(self, in_channels, n_filters, k_size,  stride, padding, bias=True, dilation=1):
        super(conv2DBatchNormRelu, self).__init__()

        self.cbr_unit = nn.Sequential(
            nn.Conv2d(
                int(in_channels), # 입력 채널의 수, rgb의 경우 3
                      int(n_filters), # 필터의 수
                      kernel_size=k_size, # 커널의 크기
                        padding=padding, 
                        stride=stride, 
                        bias=bias, # bias (default=True) : 편향 사용의 여부
                        dilation=dilation # 딜레이션 레이트 : default = 1
                ),
            nn.BatchNorm2d(int(n_filters)), # 2차원 이미지 데이터에 대한 배치 정규화
            nn.ReLU(inplace=True),
                                 ) 

    def forward(self, inputs):
        outputs = self.cbr_unit(inputs)
        return outputs


class SemanticImageEncoder(nn.Module):
    '''
    Tianyang:
    Simple Convolutional Encoder for semantic map images
    Input size: flexible
    Output size: 1/2 input
    '''

    def __init__(self, in_channels = 3, out_channels = 32):
        super(SemanticImageEncoder, self).__init__()

        self._encoder = nn.Sequential(
            conv2DBatchNormRelu(in_channels = in_channels, # 입력 채널 개수, RGB일 때 : 3
                                n_filters = 16, # 출력 채널 개수
                                k_size = 3,     # 컨볼루션 필터 크기 : (3,3) 
                                stride = 1,         # 필터 이동 간격 (default=1)
                                padding = 1),       # padding :(default=0) 
            
            conv2DBatchNormRelu(in_channels = 16, n_filters = 16, \
                k_size = 4,  stride = 1, padding = 2),
            
            nn.MaxPool2d((2, 2), stride=(2, 2)),

            conv2DBatchNormRelu(in_channels = 16, n_filters = out_channels, \
                k_size = 5,  stride = 1, padding = 2),
            )

    def forward(self, input): # 4차원 input (batchsize(frame 개수), 차원수, h, w)
        # encoded = self._encoder(input.type(torch.cuda.FloatTensor))
        encoded = self._encoder(input.type(torch.FloatTensor))
        return encoded

In [13]:
image_channels=3
embed_dim_image=32
image_encoder = SemanticImageEncoder(in_channels = image_channels, out_channels = embed_dim_image)

In [14]:
#image encoder 에는 4차원 input을 주어야 함 
# ValueError: expected 4D input (got 3D input)
# ft_img_0=image_encoder(train_dset.tensor_list[0])
# print(ft_img_0)

In [15]:
def list2batch(seq):
    # assemble a list of elements to batch, batch_idx: 0th dimension
    stacked = torch.tensor(seq[0]).unsqueeze(0)
    i = 1
    l = len(seq) # 리스트 길이 (이미지 파일 수)
    while i < l:
        stacked = torch.cat((stacked, torch.tensor(seq[i]).unsqueeze(0)), 0)
        i += 1
    return stacked

In [16]:
batch_img_list=list2batch(train_dset.tensor_list)
print(batch_img_list)
print(batch_img_list.shape)
print(type(batch_img_list))

/home/gpuadmin/anaconda3/envs/traffino/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  This is separate from the ipykernel package so we can avoid doing imports until
/home/gpuadmin/anaconda3/envs/traffino/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys


tensor([[[[0.0000, 0.0000, 0.0078,  ..., 0.0078, 0.0078, 0.0039],
          [0.0039, 0.0000, 0.0118,  ..., 0.0000, 0.0000, 0.0000],
          [0.0039, 0.0000, 0.0118,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0314, 0.1922, 0.1294,  ..., 0.2902, 0.1725, 0.0039],
          [0.0000, 0.0039, 0.0000,  ..., 0.2863, 0.1765, 0.0039],
          [0.0235, 0.0118, 0.0000,  ..., 0.2980, 0.1882, 0.0196]],

         [[0.0000, 0.0000, 0.0039,  ..., 0.0078, 0.0078, 0.0039],
          [0.0039, 0.0000, 0.0078,  ..., 0.0039, 0.0039, 0.0078],
          [0.0039, 0.0000, 0.0078,  ..., 0.0157, 0.0235, 0.0275],
          ...,
          [0.0353, 0.1961, 0.1333,  ..., 0.2824, 0.1725, 0.0039],
          [0.0039, 0.0078, 0.0039,  ..., 0.2784, 0.1765, 0.0039],
          [0.0235, 0.0118, 0.0000,  ..., 0.2902, 0.1882, 0.0196]],

         [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0039, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0039, 0.0000, 0.0000,  ..., 0

In [17]:
ft_img_769=image_encoder(batch_img_list)
print(ft_img_769)

tensor([[[[0.0000e+00, 6.8419e-01, 0.0000e+00,  ..., 6.9899e-03,
           0.0000e+00, 9.5197e-01],
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [2.1932e+00, 2.1953e+00, 0.0000e+00,  ..., 0.0000e+00,
           1.0288e-01, 0.0000e+00],
          ...,
          [2.4005e+00, 0.0000e+00, 2.7180e-01,  ..., 0.0000e+00,
           0.0000e+00, 1.0479e-01],
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00]],

         [[0.0000e+00, 0.0000e+00, 7.8761e-01,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [1.0013e-01, 0.0000e+00, 3.6137e+00,  ..., 0.0000e+00,
           2.9627e-01, 6.0187e-01],
          [0.0000e+00, 1.6036e+00, 5.5664e+00,  ..., 0.0000e+00,
           8.9287e-01, 1.5028e+00],
          ...,
          [1.0636e-01, 0.0000e+00, 0.0000e+00,  ..., 0.0000

In [19]:
print(ft_img_769.shape)
print(type(ft_img_769)) # (frame수, 32, 180, 320)
print(len(ft_img_769))

torch.Size([727, 32, 180, 320])
<class 'torch.Tensor'>
727


In [23]:
import pickle as pk

In [27]:
with open(os.path.join(data_dir, 'ft_img_769.pkl'), 'wb') as f:

    pk.dump(ft_img_769,f, protocol=4) # cannot serialize a bytes object larger than 4 GiB